<a href="https://colab.research.google.com/github/Piroon1123/choikeonwoo/blob/main/AI_06_%EC%B5%9C%EA%B1%B4%EC%9A%B0_CP2_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. 데이터 불러오기

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

df_url = '/content/drive/MyDrive/Colab Notebooks/policy_metetable.xlsx'
df = pd.read_excel(df_url)

In [ ]:
df_1 = pd.DataFrame()

In [ ]:
df['Full'] = df['서비스명'] + " " + df['선정기준']+ " " + df['서비스목적'] + " " + df['지원내용'] + " " + df['지원대상']

### 1. 신청절차

In [ ]:
def t_apply_process(dfcolumn):
  dfcolumn=dfcolumn.astype('str')
  f_list=[]
  online=['온라인','팩스','우편','이메일','전화','사이트','mail','모바일']
  offline=['방문','오프라인']
  for _ in dfcolumn:
    list=[]
    if any(str in _ for str in online):
      list.append('온라인')
    if any(str in _ for str in offline):
      list.append('오프라인')
    if len(list)==2:
      list=[]
      list.append('복합')
    if len(list)==0:
      list.append('오프라인')
    # else:
    #   list.append('오프라인')
    f_list.append(','.join(list))
  return f_list

In [ ]:
df['태그-신청절차']=t_apply_process(df['신청절차'])
df['태그-신청절차'].head()

0    오프라인
1     온라인
2    오프라인
3    오프라인
4    오프라인
Name: 태그-신청절차, dtype: object

In [ ]:
df_1['태그-신청절차'] = df['태그-신청절차']

In [ ]:
df_1['태그-신청절차'].value_counts()

오프라인    73333
복합      10894
온라인      4259
Name: 태그-신청절차, dtype: int64

In [ ]:
for i in range(len(df_1)):
  if df_1['태그-신청절차'][i] == '복합':
    df_1['태그-신청절차'][i] = 'A10-00'

  elif df_1['태그-신청절차'][i] == '온라인':
    df_1['태그-신청절차'][i] = 'A10-01'

  elif df_1['태그-신청절차'][i] == '오프라인':
    df_1['태그-신청절차'][i] = 'A10-02'

In [ ]:
# 최종확인

df_1['태그-신청절차'].value_counts()

A10-02    73333
A10-00    10894
A10-01     4259
Name: 태그-신청절차, dtype: int64

### 2. 가구원

In [ ]:
def t_house_owner(dfcolumn):
  dfcolumn=dfcolumn.astype('str')

  f_list=[]
  house=['주택','1주택','2주택','다주택','등기','재산세']
  n_house=['무주택']
  owner=['세대주','주인']
 
  for _ in dfcolumn:
    list=[]
    if any(str in _ for str in house):
      if any(str in _ for str in owner):
        list.append('주택 세대주')
      else :
        list.append('주택 세대원')

    if any(str in _ for str in n_house):
      if any(str in _ for str in owner):
        list.append('무주택 세대주')
      else :
        list.append('무주택 세대원')

    if (len(list)==4) or (len(list)==0):
      list.append('무관')

    f_list.append(','.join(list))
  
  return f_list

In [ ]:
df['태그-가구원']=t_house_owner(df['Full']+" "+df['소관기관'])
df['태그-가구원'].head()

0                무관
1            주택 세대원
2    주택 세대주,무주택 세대주
3                무관
4                무관
Name: 태그-가구원, dtype: object

In [ ]:
df_1['태그-가구원'] = df['태그-가구원']

In [ ]:
df_1['태그-가구원'].value_counts()

무관                82897
주택 세대원             4791
주택 세대원,무주택 세대원      326
주택 세대주              299
주택 세대주,무주택 세대주      173
Name: 태그-가구원, dtype: int64

In [ ]:
gagu = df['태그-가구원'].str.replace(',','/')
gagu_list = gagu.str.split('/')
gagu_data = gagu_list.apply(lambda x : pd.Series(x))
gaguframe = gagu_data.stack().reset_index(level=1, drop=True).to_frame('태그-가구원목록')

In [ ]:
gaguframe['태그-가구원목록'] = gaguframe['태그-가구원목록'].map({'무관':'A05-00', '무주택 세대주':'A05-01', '무주택 세대원':'A05-02', '주택 세대주':'A05-03', '주택 세대원':'A05-04'}, na_action=None)

In [ ]:
gaguframe.value_counts()

태그-가구원목록
A05-00      82897
A05-04       5117
A05-03        472
A05-02        326
A05-01        173
dtype: int64

In [ ]:
df_1 = df_1.merge(gaguframe, left_index=True, right_index=True, how='left')

In [ ]:
df_1 = df_1.drop(columns=['태그-가구원'])

In [ ]:
df_1

,태그-신청절차,태그-가구원목록
0,A10-02,A05-00
1,A10-01,A05-04
2,A10-02,A05-03
2,A10-02,A05-01
3,A10-02,A05-00
...,...,...
88481,A10-00,A05-00
88482,A10-00,A05-00
88483,A10-00,A05-00
88484,A10-02,A05-04


### 3. 성별

In [ ]:
def t_gender(dfcolumn):
  dfcolumn=dfcolumn.astype('str')

  f_list=[]
  male=['남자','남성', '군인', '의경', '유공자', '재향' ,'남아']
  female=['여자','여성','출산', '폭력', '산모', '임신',  '미혼모', '여아']
  for _ in dfcolumn:
    list = []
    if any(str in _ for str in male) & any(str not in _ for str in female):
      list.append('남자')
    if any(str not in _ for str in male) & any(str in _ for str in female):
      list.append('여자')
    if len(list)==2 or len(list)==0:
      list=[]
      list.append('무관')
    # else :
    #   list.append('무관')
    f_list.append(','.join(list))
  return f_list

In [ ]:
df['태그-성별']=t_gender(df['서비스명'])
df['태그-성별'].head()

0    무관
1    무관
2    무관
3    무관
4    무관
Name: 태그-성별, dtype: object

In [ ]:
df['태그-성별'].value_counts()

무관    83429
여자     4154
남자      903
Name: 태그-성별, dtype: int64

In [ ]:
df_1['태그-성별'] = df['태그-성별']

In [ ]:
df_1['태그-성별'] = df_1['태그-성별'].map({'무관':'A02-00', '남자':'A02-01', '여자':'A02-02'}, na_action=None)

In [ ]:
df_1

,태그-신청절차,태그-가구원목록,태그-성별
0,A10-02,A05-00,A02-00
1,A10-01,A05-04,A02-00
2,A10-02,A05-03,A02-00
2,A10-02,A05-01,A02-00
3,A10-02,A05-00,A02-00
...,...,...,...
88481,A10-00,A05-00,A02-00
88482,A10-00,A05-00,A02-00
88483,A10-00,A05-00,A02-02
88484,A10-02,A05-04,A02-00


### 4. 학력

In [ ]:
def t_study_type(dfcolumn):
  dfcolumn=dfcolumn.astype('str')

  f_list=[]
  
  for _ in dfcolumn:
    s_list=[]
    if '대학' in _ :
      a=_.find('대학')
      if _[a:].find('재학')>-1:
        s_list.append('대학(원)재학')
      if _[a:].find('졸업')>-1:
        s_list.append('대학(원)졸업')
      if _[a:].find('휴학')>-1:
        s_list.append('대학(원)휴학')
      # else :
      #   s_list.append('대학(원)재학')
      elif _[a:].find('재학') == -1 and _[a:].find('졸업') == -1 and _[a:].find('휴학') == -1:
        s_list.append('대학(원)재학')

      while _[a+1:].find('대학') != -1:
        a=_[a+1:].find('대학') + a +1
        if _[a:].find('재학')>0:
          s_list.append('대학(원)재학')
        if _[a:].find('졸업')>0:
          s_list.append('대학(원)졸업')
        if _[a:].find('휴학')>0:
          s_list.append('대학(원)휴학')
        # else :
        #   s_list.append('대학(원)재학')
        elif _[a:].find('재학') == -1 and _[a:].find('졸업') == -1 and _[a:].find('휴학') == -1:
          s_list.append('대학(원)재학')

    if '고등' in _ :
      s_list.append('고등학교졸업')

    else :
      s_list.append('무관')
    studylist=list(set(s_list))
    f_list.append(','.join(studylist))
  return f_list

In [ ]:
df['태그-학력']=t_study_type(df['Full'])
df['태그-학력'].head()

0    무관
1    무관
2    무관
3    무관
4    무관
Name: 태그-학력, dtype: object

In [ ]:
df['태그-학력'].value_counts()

무관                                83664
고등학교졸업                             3247
대학(원)재학,무관                          618
대학(원)재학,고등학교졸업                      451
고등학교졸업,대학(원)졸업                      218
무관,대학(원)졸업                          202
대학(원)재학,고등학교졸업,대학(원)졸업               29
대학(원)재학,무관,대학(원)졸업                   25
무관,대학(원)재학,대학(원)휴학,대학(원)졸업            9
대학(원)재학,고등학교졸업,대학(원)휴학                6
대학(원)재학,대학(원)휴학,대학(원)졸업,고등학교졸업        6
대학(원)재학,무관,대학(원)휴학                    5
무관,대학(원)휴학,대학(원)졸업                    2
무관,대학(원)휴학                            2
고등학교졸업,대학(원)휴학                        1
대학(원)휴학,대학(원)졸업,고등학교졸업                1
Name: 태그-학력, dtype: int64

In [ ]:
for i in range (len(df)):
  if df['태그-학력'][i].find('무관') != -1: # 무관 포함 시 --> 무관
    df['태그-학력'][i] = '무관'

  if len(df['태그-학력'][i]) >= 30: # 모든 학력 다 해당 --> 무관
    df['태그-학력'][i] = '무관'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df['태그-학력'].value_counts()

무관                        84533
고등학교졸업                     3247
대학(원)재학,고등학교졸업              451
고등학교졸업,대학(원)졸업              218
대학(원)재학,고등학교졸업,대학(원)졸업       29
대학(원)재학,고등학교졸업,대학(원)휴학        6
고등학교졸업,대학(원)휴학                1
대학(원)휴학,대학(원)졸업,고등학교졸업        1
Name: 태그-학력, dtype: int64

In [ ]:
df_1['태그-학력'] = df['태그-학력']

In [ ]:
study = df['태그-학력'].str.replace(',','/')
study_list = study.str.split('/')
study_data = study_list.apply(lambda x : pd.Series(x))
studyframe = study_data.stack().reset_index(level=1, drop=True).to_frame('태그-학력목록')

In [ ]:
studyframe['태그-학력목록'] = studyframe['태그-학력목록'].map({'무관':'A03-00', '고등학교졸업':'A03-01','대학(원)재학':'A03-02','대학(원)휴학':'A03-03','대학(원)졸업':'A03-04'}, na_action=None)

In [ ]:
df_1 = df_1.merge(studyframe, left_index=True, right_index=True, how='left')

In [ ]:
df_1 = df_1.drop(columns=['태그-학력'])
df_1

,태그-신청절차,태그-가구원목록,태그-성별,태그-학력목록
0,A10-02,A05-00,A02-00,A03-00
1,A10-01,A05-04,A02-00,A03-00
2,A10-02,A05-03,A02-00,A03-00
2,A10-02,A05-01,A02-00,A03-00
3,A10-02,A05-00,A02-00,A03-00
...,...,...,...,...
88481,A10-00,A05-00,A02-00,A03-00
88482,A10-00,A05-00,A02-00,A03-00
88483,A10-00,A05-00,A02-02,A03-00
88484,A10-02,A05-04,A02-00,A03-00


### 5. 결혼

In [ ]:
def t_marriage(dfcolumn):
  dfcolumn=dfcolumn.astype('str')

  f_list=[]

  n_marriage=['미혼','한부모']
  marriage=['부부', '신혼', '결혼', '가족', '배우자','남편','부인','아내']
  b_marriage=['파혼','이혼']
  re_marriage=['재혼']
  
  for _ in dfcolumn:
   if any(str in _ for str in n_marriage) :
     f_list.append('A06-01')
   elif any(str in _ for str in marriage) :
     f_list.append('A06-02')
   elif any(str in _ for str in b_marriage) :
     f_list.append('A06-03')
   elif any(str in _ for str in re_marriage) :
     f_list.append('A06-04')
   else :
     f_list.append('A06-00')
  return f_list

In [ ]:
df['태그-결혼']=t_marriage(df['Full'])
df['태그-결혼'].head()

0    A06-02
1    A06-02
2    A06-02
3    A06-00
4    A06-00
Name: 태그-결혼, dtype: object

In [ ]:
df['태그-결혼'].value_counts()

A06-00    66257
A06-02    17375
A06-01     4838
A06-03       16
Name: 태그-결혼, dtype: int64

In [ ]:
df_1['태그-결혼'] = df['태그-결혼']

In [ ]:
df_1

,태그-신청절차,태그-가구원목록,태그-성별,태그-학력목록,태그-결혼
0,A10-02,A05-00,A02-00,A03-00,A06-02
1,A10-01,A05-04,A02-00,A03-00,A06-02
2,A10-02,A05-03,A02-00,A03-00,A06-02
2,A10-02,A05-01,A02-00,A03-00,A06-02
3,A10-02,A05-00,A02-00,A03-00,A06-00
...,...,...,...,...,...
88481,A10-00,A05-00,A02-00,A03-00,A06-00
88482,A10-00,A05-00,A02-00,A03-00,A06-02
88483,A10-00,A05-00,A02-02,A03-00,A06-02
88484,A10-02,A05-04,A02-00,A03-00,A06-02


### 6. 관심상황특성

In [ ]:
def t_interest(dfcolumn1, dfcolumn2):
  dfcolumn1 = dfcolumn1.astype('str') # 지원대상
  dfcolumn2 = dfcolumn2.astype('str') # 지원내용

  company = ['기업', '업체', '창업', '경영']
  finance = ['금융', '자금', '대출', '융자', '상환', '보험료']
  education = ['교육', '회의', '치료', '상담', '훈련', '컨설팅', '성장', '커리어', '세미나']
  marriage = ['결혼', '임신', '신혼', '출산', '산모']
  retire = ['은퇴', '퇴직']
  army = ['병력', '예비군', '입영', '의무경찰', '입대', '군복무자', '군입대', '전역']
  violation=['폭력','범죄','중독','마약','알콜','구타','폭언']
  traffic=['자동차','교통','충전소','주유소','대중교통','운전자']

  f_list = []

  for a, b in zip(dfcolumn1, dfcolumn2):
    list = []
    if any(str in a for str in company):
      if any(str in b for str in finance):
        if any(str in b for str in education):
          list.append('A13-06,A13-05')

        else :
          list.append('A13-06')

      elif any(str not in b for str in finance):
        list.append('A13-05')

    elif any(str not in a for str in company):
      
      if any(str in b for str in marriage):
        list.append('A13-01')

      if any(str in a for str in retire):
        list.append('A13-02')

      if any(str in b for str in army):
        list.append('A13-00')

      if any(str in b for str in violation): 
        list.append('A13-07')
      
      if any(str in b for str in traffic): 
        list.append('A13-08')
      
      elif any(str in b for str in finance):
        if any(str in b for str in education):
          list.append('A13-04,A13-03')

        else :
          list.append('A13-04')

      elif any(str not in b for str in finance):
        list.append('A13-03')
    
    f_list.append(','.join(list))

  return f_list

In [ ]:
df['태그-관심상황특성']=t_interest(df['지원대상'], df['지원내용'])
df['태그-관심상황특성'].head()

0    A13-01,A13-03
1           A13-03
2           A13-04
3           A13-06
4           A13-03
Name: 태그-관심상황특성, dtype: object

In [ ]:
df['태그-관심상황특성'].value_counts()

A13-03                         38618
A13-01,A13-03                  19181
A13-05                         10768
A13-04                          4756
A13-06                          4385
A13-08                          4176
A13-07,A13-03                   1530
A13-04,A13-03                   1298
A13-01,A13-04,A13-03            1258
A13-06,A13-05                    742
A13-01,A13-07,A13-03             493
A13-01,A13-08                    448
A13-07,A13-08                    222
A13-01,A13-07,A13-04,A13-03      201
A13-07,A13-04,A13-03             132
A13-00,A13-03                     99
A13-01,A13-04                     44
A13-02,A13-03                     35
A13-02,A13-04                     30
A13-00,A13-08                     14
A13-01,A13-00,A13-03               8
A13-07,A13-04                      7
A13-01,A13-07,A13-08               7
A13-01,A13-02,A13-03               6
A13-00,A13-04                      5
A13-02,A13-08                      4
A13-02,A13-04,A13-03               4
A

In [ ]:
df_1['태그-관심상황특성'] = df['태그-관심상황특성']

In [ ]:
interest = df['태그-관심상황특성'].str.replace(',','/')
interest_list = interest.str.split('/')
interest_data = interest_list.apply(lambda x : pd.Series(x))
interestframe = interest_data.stack().reset_index(level=1, drop=True).to_frame('태그-관심상황목록')

In [ ]:
interestframe.value_counts()

태그-관심상황목록
A13-03       62874
A13-01       21653
A13-05       11510
A13-04        7743
A13-06        5127
A13-08        4874
A13-07        2597
A13-00         140
A13-02          81
dtype: int64

In [ ]:
df_1 = df_1.merge(interestframe, left_index=True, right_index=True, how='left')

In [ ]:
df_1 = df_1.drop(columns=['태그-관심상황특성'])
df_1

,태그-신청절차,태그-가구원목록,태그-성별,태그-학력목록,태그-결혼,태그-관심상황목록
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-03
1,A10-01,A05-04,A02-00,A03-00,A06-02,A13-03
2,A10-02,A05-03,A02-00,A03-00,A06-02,A13-04
2,A10-02,A05-01,A02-00,A03-00,A06-02,A13-04
...,...,...,...,...,...,...
88482,A10-00,A05-00,A02-00,A03-00,A06-02,A13-03
88483,A10-00,A05-00,A02-02,A03-00,A06-02,A13-03
88484,A10-02,A05-04,A02-00,A03-00,A06-02,A13-01
88484,A10-02,A05-04,A02-00,A03-00,A06-02,A13-03


### 7. 지역(시도 - 시군구)

#### i) 필요한 것 설치

In [ ]:
# 지역(시도) & 지역(시군구) 코드 데이터

seoul_url = '/content/drive/MyDrive/Colab Notebooks/city/seoul.xlsx'
seoul = pd.read_excel(seoul_url)
busan_url = '/content/drive/MyDrive/Colab Notebooks/city/busan.xlsx'
busan = pd.read_excel(busan_url)
ccn_url = '/content/drive/MyDrive/Colab Notebooks/city/ccn.xlsx'
ccn = pd.read_excel(ccn_url)
ccs_url = '/content/drive/MyDrive/Colab Notebooks/city/ccs.xlsx'
ccs = pd.read_excel(ccs_url)
daegu_url = '/content/drive/MyDrive/Colab Notebooks/city/daegu.xlsx'
daegu = pd.read_excel(daegu_url)
gangwon_url = '/content/drive/MyDrive/Colab Notebooks/city/gangwon.xlsx'
gangwon = pd.read_excel(gangwon_url)
gyeongi_url = '/content/drive/MyDrive/Colab Notebooks/city/gyeongi.xlsx'
gyeongi = pd.read_excel(gyeongi_url)
incheon_url = '/content/drive/MyDrive/Colab Notebooks/city/incheon.xlsx'
incheon = pd.read_excel(incheon_url)
jrn_url = '/content/drive/MyDrive/Colab Notebooks/city/jrn.xlsx'
jrn = pd.read_excel(jrn_url)
jrs_url = '/content/drive/MyDrive/Colab Notebooks/city/jrs.xlsx'
jrs = pd.read_excel(jrs_url)
ksn_url = '/content/drive/MyDrive/Colab Notebooks/city/ksn.xlsx'
ksn = pd.read_excel(ksn_url)
kss_url = '/content/drive/MyDrive/Colab Notebooks/city/kss.xlsx'
kss = pd.read_excel(kss_url)
gwangju_url = '/content/drive/MyDrive/Colab Notebooks/city/gwangju.xlsx'
gwangju = pd.read_excel(gwangju_url)
daejun_url = '/content/drive/MyDrive/Colab Notebooks/city/daejun.xlsx'
daejun = pd.read_excel(daejun_url)
ulsan_url = '/content/drive/MyDrive/Colab Notebooks/city/ulsan.xlsx'
ulsan = pd.read_excel(ulsan_url)

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt

okt = Okt()

In [ ]:
def testing_noun(text):
  non = okt.nouns(text)

  return non

#### ii) 지역(시도) 전처리

In [ ]:
s1 = df['소관기관'].apply(testing_noun)

for i in range(len(df)):
  
  if len(s1[i]) <= 1 : # row의 단어가 한개일때
    if s1[i][0].endswith('도'):
      if s1[i][0].startswith('태권'):
        df['태그-지역(시도)'][i] = '전국'
      else:
        df['태그-지역(시도)'][i] = s1[i][0]

    elif s1[i][0].endswith('광역시') or s1[i][0].endswith('특별시'):
      df['태그-지역(시도)'][i] = s1[i][0]

    else:
      df['태그-지역(시도)'][i] = '전국'


  elif len(s1[i]) > 1: # row의 단어가 여러개 일때
    if s1[i][0].endswith('도'):
      if s1[i][0].startswith('태권'):
        df['태그-지역(시도)'][i] = '전국'
      else :
        df['태그-지역(시도)'][i] = s1[i][0]

    elif s1[i][0].endswith('광역시') or s1[i][0].endswith('특별시'):
      df['태그-지역(시도)'][i] = s1[i][0]

    elif s1[i][0].endswith('구') or s1[i][0].startswith('서울'):
      df['태그-지역(시도)'][i] = '서울특별시'

    elif s1[i][0].startswith('시흥') or s1[i][0].startswith('의왕') or s1[i][0].startswith('가평') or s1[i][0].startswith('화성') or s1[i][0].startswith('양주') or s1[i][0].startswith('파주') or s1[i][0].startswith('안성'):
      df['태그-지역(시도)'][i] = '경기도'

    elif s1[i][0].startswith('강릉'):
      df['태그-지역(시도)'][i] = '강원도'

    elif s1[i][0].startswith('청주'):
      df['태그-지역(시도)'][i] = '충청북도'

    elif s1[i][0].startswith('인천') or s1[i][0].startswith('강화') :
      df['태그-지역(시도)'][i] = '인천광역시'

    elif s1[i][0].startswith('부산'):
      df['태그-지역(시도)'][i] = '부산광역시'

    elif s1[i][1] == '광역시' or s1[i][1] == '특별자치시' or s1[i][1] == '특별자치도':
      df['태그-지역(시도)'][i] = s1[i][0] + s1[i][1]

    else :
      df['태그-지역(시도)'][i] = '전국'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarn

In [ ]:
df['태그-지역(시도)'].value_counts()

경기도        9888
경상북도       8937
경상남도       8134
전라남도       7953
전국         7171
강원도        6834
서울특별시      6505
충청남도       5891
전라북도       5120
충청북도       4863
부산광역시      4748
인천광역시      2983
대구광역시      2727
대전광역시      2100
광주광역시      1920
울산광역시      1907
제주특별자치도     491
세종특별자치시     314
Name: 태그-지역(시도), dtype: int64

#### iii) 지역(시군구) 전처리

In [ ]:
for i in range(len(df)):
  
  if df['태그-지역(시도)'][i] == '전국' or len(s1[i]) < 1:
    df['태그-지역(시군구)'][i] = ''

  elif len(s1[i]) == 1:
    if df['태그-지역(시도)'][i].endswith('광역시') or df['태그-지역(시도)'][i].endswith('북도') or df['태그-지역(시도)'][i].endswith('남도'):
      df['태그-지역(시군구)'][i] = '전체'

    else :
      df['태그-지역(시군구)'][i] = ''

  elif len(s1[i]) ==2:

    if s1[i][1].endswith('시') or s1[i][1].endswith('군') or s1[i][1].endswith('구'):
      if s1[i][1].endswith('특별자치시') :
        df['태그-지역(시군구)'][i] = '세종시 전체'
      elif s1[i][1].endswith('특별자치도') :
        df['태그-지역(시군구)'][i] = '제주시'
      elif s1[i][1].endswith('광역시') or s1[i][1].endswith('도시') or s1[i][1].endswith('대구') or s1[i][1].endswith('대전') or s1[i][1].startswith('강릉원주대') or s1[i][1].endswith('서울특별시'):
        df['태그-지역(시군구)'][i] = '전체'
      else:
        df['태그-지역(시군구)'][i] = s1[i][1]

    elif s1[i][0].endswith('시') or s1[i][0].endswith('군') or s1[i][0].endswith('구'):
      if s1[i][0].endswith('특별자치시'):
        df['태그-지역(시군구)'][i] = '세종시 전체'
      elif s1[i][0].endswith('특별자치도') :
        df['태그-지역(시군구)'][i] = '제주시'
      elif s1[i][0].endswith('광역시') or s1[i][0].endswith('도시') or s1[i][0].endswith('대구') or s1[i][1].endswith('대전') or s1[i][0].startswith('강릉원주대') or s1[i][0].endswith('서울특별시'):
        df['태그-지역(시군구)'][i] = '전체'
      else:
        df['태그-지역(시군구)'][i] = s1[i][0]

    elif s1[i][0].startswith('시흥') or s1[i][0].startswith('의왕') or s1[i][0].startswith('가평') or s1[i][0].startswith('화성') or s1[i][0].startswith('강화') or s1[i][0].startswith('양주') or s1[i][0].startswith('파주') or s1[i][0].startswith('안성'):
      df['태그-지역(시군구)'][i] = s1[i][0] + '시'

    elif s1[i][0].startswith('강릉'):
      df['태그-지역(시군구)'][i] = '강릉시'

    elif s1[i][0].startswith('청주'):
      df['태그-지역(시군구)'][i] = '청주시'

    else :
      df['태그-지역(시군구)'][i] = '전체'


  elif len(s1[i]) >= 3 :

    if s1[i][2].endswith('시') or s1[i][2].endswith('군') or s1[i][2].endswith('구'):
      if s1[i][1].startswith('미추홀'):
        df['태그-지역(시군구)'][i] = s1[i][1] + '구'
      elif s1[i][2].endswith('도시'):
        df['태그-지역(시군구)'][i] = '전체'
      else :
        df['태그-지역(시군구)'][i] = s1[i][2]

    elif s1[i][1].endswith('시') or s1[i][1].endswith('군') or s1[i][1].endswith('구'):
      if s1[i][1].endswith('특별자치시') :
        df['태그-지역(시군구)'][i] = '세종시 전체'
      elif s1[i][1].endswith('특별자치도') :
        df['태그-지역(시군구)'][i] = '제주시'
      elif s1[i][1].endswith('광역시') or s1[i][1].endswith('도시') or s1[i][1].endswith('대구') or s1[i][1].endswith('대전') or s1[i][1].startswith('강릉원주대') or s1[i][1].endswith('서울특별시'):
        df['태그-지역(시군구)'][i] = '전체'
      else:  
        df['태그-지역(시군구)'][i] = s1[i][1]

    elif s1[i][0].endswith('시') or s1[i][0].endswith('군') or s1[i][0].endswith('구'):
      if s1[i][0].endswith('특별자치시') :
        df['태그-지역(시군구)'][i] = '세종시 전체'
      elif  s1[i][0].endswith('특별자치도'):
        df['태그-지역(시군구)'][i] = '제주시'
      elif s1[i][0].endswith('광역시') or s1[i][0].endswith('도시') or s1[i][0].endswith('대구') or s1[i][0].startswith('강릉원주대') or s1[i][0].endswith('서울특별시'):
        df['태그-지역(시군구)'][i] = '전체'
      else:
        df['태그-지역(시군구)'][i] = s1[i][0]

    elif s1[i][0].startswith('시흥') or s1[i][0].startswith('의왕') or s1[i][0].startswith('가평') or s1[i][0].startswith('화성') or s1[i][0].startswith('양주') or s1[i][0].startswith('파주') or s1[i][0].startswith('안성'):
      df['태그-지역(시군구)'][i] = s1[i][0] + '시'

    elif s1[i][0].startswith('강릉'):
      df['태그-지역(시군구)'][i] = '강릉시'

    elif s1[i][0].startswith('청주'):
      df['태그-지역(시군구)'][i] = '청주시'

    elif s1[i][0].startswith('강화'):
      df['태그-지역(시군구)'][i] = '강화군'

    else :
      df['태그-지역(시군구)'][i] = '전체'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingW

In [ ]:
df['태그-지역(시군구)'].value_counts()

       8279
전체     6938
동구     1726
중구     1592
서구     1476
       ... 
성북구     198
옹진군     196
도봉구     192
강남구     191
강동구     182
Name: 태그-지역(시군구), Length: 205, dtype: int64

In [ ]:
for ok in range(len(df)):

  if df['태그-지역(시도)'][ok] == '전국' :
    df['태그-지역(시도)'][ok] = 'A01-00'

  elif df['태그-지역(시도)'][ok] == '서울특별시' :
    df['태그-지역(시도)'][ok] = 'A01-01'

    for a in range (len(seoul)):
      if df['태그-지역(시군구)'][ok] == seoul['city'][a]:
        df['태그-지역(시군구)'][ok] = seoul['num'][a]

  elif df['태그-지역(시도)'][ok] == '부산광역시' :
    df['태그-지역(시도)'][ok] = 'A01-02'

    for b in range (len(busan)):
      if df['태그-지역(시군구)'][ok] == busan['city'][b]:
        df['태그-지역(시군구)'][ok] = busan['num'][b]

  elif df['태그-지역(시도)'][ok] == '대구광역시' :
    df['태그-지역(시도)'][ok] = 'A01-03'

    for c in range (len(daegu)):
      if df['태그-지역(시군구)'][ok] == daegu['city'][c]:
        df['태그-지역(시군구)'][ok] = daegu['num'][c]

  elif df['태그-지역(시도)'][ok] == '인천광역시' :
    df['태그-지역(시도)'][ok] = 'A01-04'

    for d in range (len(incheon)):
      if df['태그-지역(시군구)'][ok] == incheon['city'][d]:
        df['태그-지역(시군구)'][ok] = incheon['num'][d]

  elif df['태그-지역(시도)'][ok] == '광주광역시' :
    df['태그-지역(시도)'][ok] = 'A01-05'

    for e in range (len(gwangju)):
      if df['태그-지역(시군구)'][ok] == gwangju['city'][e]:
        df['태그-지역(시군구)'][ok] = gwangju['num'][e]


  elif df['태그-지역(시도)'][ok] == '대전광역시' :
    df['태그-지역(시도)'][ok] = 'A01-06'

    for f in range (len(daejun)):
      if df['태그-지역(시군구)'][ok] == daejun['city'][f]:
        df['태그-지역(시군구)'][ok] = daejun['num'][f]

  elif df['태그-지역(시도)'][ok] == '울산광역시' :
    df['태그-지역(시도)'][ok] = 'A01-07'

    for g in range (len(ulsan)):
      if df['태그-지역(시군구)'][ok] == ulsan['city'][g]:
        df['태그-지역(시군구)'][ok] = ulsan['num'][g]

  elif df['태그-지역(시도)'][ok] == '세종특별자치시' :
    df['태그-지역(시도)'][ok] = 'A01-08'
    df['태그-지역(시군구)'][ok] = 0

  elif df['태그-지역(시도)'][ok] == '경기도' :
    df['태그-지역(시도)'][ok] = 'A01-09'

    for h in range (len(gyeongi)):
      if df['태그-지역(시군구)'][ok] == gyeongi['city'][h]:
        df['태그-지역(시군구)'][ok] = gyeongi['num'][h]

  elif df['태그-지역(시도)'][ok] == '강원도' :
    df['태그-지역(시도)'][ok] = 'A01-10'

    for i in range (len(gangwon)):
      if df['태그-지역(시군구)'][ok] == gangwon['city'][i]:
        df['태그-지역(시군구)'][ok] = gangwon['num'][i]

  elif df['태그-지역(시도)'][ok] == '충청북도' :
    df['태그-지역(시도)'][ok] = 'A01-11'

    for j in range (len(ccn)):
      if df['태그-지역(시군구)'][ok] == ccn['city'][j]:
        df['태그-지역(시군구)'][ok] = ccn['num'][j]

  elif df['태그-지역(시도)'][ok] == '충청남도' :
    df['태그-지역(시도)'][ok] = 'A01-12'

    for k in range (len(ccs)):
      if df['태그-지역(시군구)'][ok] == ccs['city'][k]:
        df['태그-지역(시군구)'][ok] = ccs['num'][k]

  elif df['태그-지역(시도)'][ok] == '전라북도' :
    df['태그-지역(시도)'][ok] = 'A01-13'

    for l in range (len(jrn)):
      if df['태그-지역(시군구)'][ok] == jrn['city'][l]:
        df['태그-지역(시군구)'][ok] = jrn['num'][l]

  elif df['태그-지역(시도)'][ok] == '전라남도' :
    df['태그-지역(시도)'][ok] = 'A01-14'

    for m in range (len(jrs)):
      if df['태그-지역(시군구)'][ok] == jrs['city'][m]:
        df['태그-지역(시군구)'][ok] = jrs['num'][m]

  elif df['태그-지역(시도)'][ok] == '경상북도' :
    df['태그-지역(시도)'][ok] = 'A01-15'

    for n in range (len(ksn)):
      if df['태그-지역(시군구)'][ok] == ksn['city'][n]:
        df['태그-지역(시군구)'][ok] = ksn['num'][n]

  elif df['태그-지역(시도)'][ok] == '경상남도' :
    df['태그-지역(시도)'][ok] = 'A01-16'

    for o in range (len(kss)):
      if df['태그-지역(시군구)'][ok] == kss['city'][o]:
        df['태그-지역(시군구)'][ok] = kss['num'][o]

  elif df['태그-지역(시도)'][ok] == '제주특별자치도' :
    df['태그-지역(시도)'][ok] = 'A01-17'

    if df['태그-지역(시군구)'][ok] == '제주시' or df['태그-지역(시군구)'][ok] == '전체':
      df['태그-지역(시군구)'][ok] = 0

    elif df['태그-지역(시군구)'][ok] == '서귀포시':
      df['태그-지역(시군구)'][ok] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: Setti

In [ ]:
for i in range (len(df)):
  if len(str(df['태그-지역(시군구)'][i])) == 1:
    df['태그-지역(시군구)'][i] = '0' + str(df['태그-지역(시군구)'][i])

  elif len(str(df['태그-지역(시군구)'][i])) > 1:
    df['태그-지역(시군구)'][i] = str(df['태그-지역(시군구)'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df['태그-지역(시도)'].value_counts()

A01-09    9888
A01-15    8937
A01-16    8134
A01-14    7953
A01-00    7171
A01-10    6834
A01-01    6505
A01-12    5891
A01-13    5120
A01-11    4863
A01-02    4748
A01-04    2983
A01-03    2727
A01-06    2100
A01-05    1920
A01-07    1907
A01-17     491
A01-08     314
Name: 태그-지역(시도), dtype: int64

In [ ]:
df['태그-지역(시군구)'].value_counts()

      8279
00    7252
01    5177
02    5104
03    4987
04    4879
05    4877
08    4059
06    3973
07    3891
09    3492
10    3461
11    3256
14    3100
12    2918
13    2553
15    2442
18    2278
16    2209
17    1918
19    1404
20    1224
21    1146
22    1146
23     697
24     497
25     442
31     398
30     316
27     311
28     283
29     282
26     235
Name: 태그-지역(시군구), dtype: int64

In [ ]:
area1 = pd.DataFrame(df['태그-지역(시도)'])
area1 = area1.rename(columns={'태그-지역(시도)' : '시도번호'})
area2 = pd.DataFrame(df['태그-지역(시군구)'])
area2 = area2.rename(columns={'태그-지역(시군구)' : '시군구번호'})

In [ ]:
areas = pd.concat([area1, area2], axis=1)
areas

,시도번호,시군구번호
0,A01-00,
1,A01-00,
2,A01-00,
3,A01-00,
4,A01-00,
...,...,...
88481,A01-13,00
88482,A01-17,00
88483,A01-12,03
88484,A01-11,03


In [ ]:
areas['지역_number'] = areas['시도번호'] # full_number 칸을 만들기 위한 임시방편

In [ ]:
for i in range (len(areas)):

  if len(str(areas['시군구번호'][i])) == 0 :
    areas['지역_number'][i] = areas['시도번호'][i]

  elif len(str(areas['시군구번호'][i])) > 0:
    areas['지역_number'][i] = areas['시도번호'][i] + '-' + areas['시군구번호'][i]

In [ ]:
area3 = areas['지역_number']

In [ ]:
df_1 = df_1.merge(area3, left_index=True, right_index=True, how='left')

In [ ]:
df_1

,태그-신청절차,태그-가구원목록,태그-성별,태그-학력목록,태그-결혼,태그-관심상황목록,지역_number
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-03,A01-00
1,A10-01,A05-04,A02-00,A03-00,A06-02,A13-03,A01-00
2,A10-02,A05-03,A02-00,A03-00,A06-02,A13-04,A01-00
2,A10-02,A05-01,A02-00,A03-00,A06-02,A13-04,A01-00
...,...,...,...,...,...,...,...
88482,A10-00,A05-00,A02-00,A03-00,A06-02,A13-03,A01-17-00
88483,A10-00,A05-00,A02-02,A03-00,A06-02,A13-03,A01-12-03
88484,A10-02,A05-04,A02-00,A03-00,A06-02,A13-01,A01-11-03
88484,A10-02,A05-04,A02-00,A03-00,A06-02,A13-03,A01-11-03


### 8. 자녀

#### i) 자녀 전처리

In [ ]:
def t_children(dfcolumn):
  dfcolumn=dfcolumn.astype('str')

  f_list=[]
  children=['보호자','어린이집', '보육원','어린이','아동', '출산', '임산부' , '부모', '손자녀','자녀','학비', '학자금', '아동', '영유아','영아','유아', '초등','중학' ,'중등','고등','임신', '대학', '학자', '성인 자녀', '성인자녀', '증여']
  for _ in dfcolumn:
   if any(str in _ for str in children):
     f_list.append('A07-01')
   elif any(str in _ for str in children) == False:
     f_list.append('A07-00')
  return f_list

In [ ]:
df['태그-자녀']=t_children(df['지원대상'])
df['태그-자녀'].head()

0    A07-01
1    A07-01
2    A07-00
3    A07-00
4    A07-00
Name: 태그-자녀, dtype: object

In [ ]:
df['태그-자녀'].value_counts()

A07-00    60116
A07-01    28370
Name: 태그-자녀, dtype: int64

In [ ]:
df_1['태그-자녀'] = df['태그-자녀']

In [ ]:
df_1['태그-자녀'].value_counts()

A07-00    70170
A07-01    48443
Name: 태그-자녀, dtype: int64

#### ii) 자녀상세 전처리

In [ ]:
def t_siblings(dfcolumn):

  dfcolumn=dfcolumn.astype('str')

  baby=['영아','유아', '아동']
  kid=['어린이집', '보육원']
  kinder=['유치원']
  student=['초등', '중학', '고등', '학생']
  college=['대학', '학자', '성인 자녀', '성인자녀', '증여']

  f_list=[]
  for _ in dfcolumn:
    list=[]
    if any(str in _ for str in baby):
      list.append('00')
    if any(str in _ for str in kid):
      list.append('01')
    if any(str in _ for str in kinder):
      list.append('02')
    if any(str in _ for str in student):
      list.append('03')
    if any(str in _ for str in college):
      list.append('04')
    if len(list)==0 or len(list) >= 4:
      list.append('')
    f_list.append(','.join(list))
  return f_list

In [ ]:
df['태그-자녀상세'] = t_siblings(df['지원대상'])

In [ ]:
df['태그-자녀상세'].value_counts()

                   67680
00                  7723
03                  5334
00,03               2002
01                  1729
00,01               1089
03,04                656
00,01,02             636
04                   580
00,01,03             225
00,01,02,04,         174
02,03                128
00,01,02,03,          90
00,03,04              90
01,02,03              88
00,02                 73
00,02,03              61
01,02                 38
00,04                 36
01,03                 22
02                    11
00,01,02,03,04,        8
02,03,04               6
00,01,04               2
01,02,03,04,           2
00,01,03,04,           1
01,02,04               1
00,02,04               1
Name: 태그-자녀상세, dtype: int64

In [ ]:
sibling = df['태그-자녀상세'].str.replace(',','/')
sibling_list = sibling.str.split('/')
sibling_data = sibling_list.apply(lambda x : pd.Series(x))
siblingframe = sibling_data.stack().reset_index(level=1, drop=True).to_frame('태그-자녀상세목록')

In [ ]:
df_1 = df_1.merge(siblingframe, left_index=True, right_index=True, how='left')

In [ ]:
# 번호 합쳐주는 함수

def combine(col_1, col_2):
  result = col_1
  if col_2 != '':
    result += "-" + str(col_2)
  
  return result

In [ ]:
df_1['자녀_number'] = df_1.apply(lambda x: combine(x['태그-자녀'], x['태그-자녀상세목록']), axis=1)

In [ ]:
df_1

,태그-신청절차,태그-가구원목록,태그-성별,태그-학력목록,태그-결혼,태그-관심상황목록,지역_number,태그-자녀,태그-자녀상세목록,자녀_number
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,00,A07-01-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,01,A07-01-01
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,03,A07-01-03
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-03,A01-00,A07-01,00,A07-01-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-03,A01-00,A07-01,01,A07-01-01
...,...,...,...,...,...,...,...,...,...,...
88482,A10-00,A05-00,A02-00,A03-00,A06-02,A13-03,A01-17-00,A07-01,00,A07-01-00
88483,A10-00,A05-00,A02-02,A03-00,A06-02,A13-03,A01-12-03,A07-01,,A07-01
88484,A10-02,A05-04,A02-00,A03-00,A06-02,A13-01,A01-11-03,A07-01,00,A07-01-00
88484,A10-02,A05-04,A02-00,A03-00,A06-02,A13-03,A01-11-03,A07-01,00,A07-01-00


### 9. 대상특성

#### i) 대상특성 전처리

In [ ]:
# 전처리 (1)

def t_target_ch(dfcolumn):
  dfcolumn=dfcolumn.astype('str')

  f_list=[]
  
  tc1=['장애인']
  tc2=['농축수산인','농업','축산','수산','어업','농수산']
  tc3=['질병','부상','질환']
  tc4=['난임']
  tc5=['임신','임산']
  tc6=['출산']
  tc7=['국가유공']
  tc8=['제대 군인','제대군인','제대한']
  tc9=['다문화']
  tc10=['북한이탈','북한 이탈']
  tc11=['한부모','조손']
  tc12=['1인가구','1인 가구']
  tc13=['다자녀']
  tc14=['독거노인','독거 노인']
  tc15=['입양가정','입양 가정']
  tc16=['외국인','재외국인']
  tc17=['한센']
  tc18=['이산가족','이산 가족']
  tc19=['중소','중견','기업','법인','설립','시설','기관','업체']

  for _ in dfcolumn:
    list=[]
    if any(str in _ for str in tc1):
      list.append('A12-01')
    if any(str in _ for str in tc2):
      list.append('A12-02')
    if any(str in _ for str in tc3):
      list.append('A12-03')
    if any(str in _ for str in tc4):
      list.append('A12-04')
    if any(str in _ for str in tc5):
      list.append('A12-05')
    if any(str in _ for str in tc6):
      list.append('A12-06')
    if any(str in _ for str in tc7):
      list.append('A12-07')
    if any(str in _ for str in tc8):
      list.append('A12-08')
    if any(str in _ for str in tc9):
      list.append('A12-09')
    if any(str in _ for str in tc10):
      list.append('A12-10')
    if any(str in _ for str in tc11):
      list.append('A12-11')
    if any(str in _ for str in tc12):
      list.append('A12-12')
    if any(str in _ for str in tc13):
      list.append('A12-13')
    if any(str in _ for str in tc14):
      list.append('A12-14')
    if any(str in _ for str in tc15):
      list.append('A12-15')
    if any(str in _ for str in tc16):
      list.append('A12-16')
    if any(str in _ for str in tc17):
      list.append('A12-17')
    if any(str in _ for str in tc18):
      list.append('A12-18')
    if any(str in _ for str in tc19):
      list.append('A12-19')
   
    if len(list)==0:
      list.append('A12-00')
    f_list.append(','.join(list))
  return f_list

In [ ]:
df['태그-대상특성'] = t_target_ch(df['지원대상'])

#### ii) 대상특성상세 전처리

In [ ]:
# 전처리 (2)

def t_target_ch_detail(dfcolumn):

  dfcolumn=dfcolumn.astype('str')
  dfcolumn2=dfcolumn['서비스목적'] + " " +dfcolumn['지원내용'] + " " + dfcolumn['지원대상']

  f_list=[]
  for _ in range(0,dfcolumn.shape[0]):

    list=[]

    if 'A12-00' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-00')  

    if 'A12-01' in dfcolumn['태그-대상특성'][_]:
      if any(str in dfcolumn2[_] for str in ['심한','중증']):
        list.append('A12-01-00')
      if any(str in dfcolumn2[_] for str in ['심한','중증']) == False:
        list.append('A12-01-01')
    
    if 'A12-02' in dfcolumn['태그-대상특성'][_]:
      if any(str in dfcolumn2[_] for str in ['경관']):
        list.append('A12-02-00')
      if any(str in dfcolumn2[_] for str in ['경종']):
        list.append('A12-02-01')
      if any(str in dfcolumn2[_] for str in ['과수','과일']):
        list.append('A12-02-02')
      if any(str in dfcolumn2[_] for str in ['원예','특용']):
        list.append('A12-02-03')
      if any(str in dfcolumn2[_] for str in ['농지소유','농지 소유','농지']):
        list.append('A12-02-04')
      if any(str in dfcolumn2[_] for str in ['경관', '경종', '과수', '과일', '원예','특용', '농지소유','농지 소유','농지', '연근해','구획', '양식업', '어선', '기타어업', '축산', '임업']) == False:
        list.append('A12-02-05')
      if any(str in dfcolumn2[_] for str in ['연근해','구획']):
        list.append('A12-02-06')
      if any(str in dfcolumn2[_] for str in ['양식업']):
        list.append('A12-02-07')
      if any(str in dfcolumn2[_] for str in ['어선']):
        list.append('A12-02-08')
      if any(str in dfcolumn2[_] for str in ['기타어업']):
        list.append('A12-02-09')
      if any(str in dfcolumn2[_] for str in ['축산']):
        list.append('A12-02-10')
      if any(str in dfcolumn2[_] for str in ['임업']):
        list.append('A12-02-11')

    if 'A12-03' in dfcolumn['태그-대상특성'][_]:
      if any(str in dfcolumn2[_] for str in ['요양','치매']) == False:
        list.append('A12-03-00')
      if any(str in dfcolumn2[_] for str in ['요양','치매']):
        list.append('A12-03-01')
    
    if 'A12-04' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-04')

    if 'A12-05' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-05')

    if 'A12-06' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-06')

    if 'A12-07' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-07')

    if 'A12-08' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-08')

    if 'A12-09' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-09')

    if 'A12-10' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-10')

    if 'A12-11' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-11')

    if 'A12-12' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-12')

    if 'A12-13' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-13')

    if 'A12-14' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-14')

    if 'A12-15' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-15')

    if 'A12-16' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-16')

    if 'A12-17' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-17')

    if 'A12-18' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-18')

    if 'A12-19' in dfcolumn['태그-대상특성'][_]:
      list.append('A12-19')

    f_list.append(','.join(list))
  return f_list

In [ ]:
iii = df[['서비스목적','지원내용','지원대상','태그-대상특성']]

In [ ]:
df['태그-대상특성상세'] = t_target_ch_detail(iii)

In [ ]:
chas = df['태그-대상특성상세'].str.replace(',','/')
chas_list = chas.str.split('/')
chas_data = chas_list.apply(lambda x : pd.Series(x))
chasframe = chas_data.stack().reset_index(level=1, drop=True).to_frame('태그-대상특성상세목록')

In [ ]:
df_1 = df_1.merge(chasframe, left_index=True, right_index=True, how='left')

In [ ]:
df_1

,태그-신청절차,태그-가구원목록,태그-성별,태그-학력목록,태그-결혼,태그-관심상황목록,지역_number,태그-자녀,태그-자녀상세목록,자녀_number,태그-대상특성상세목록
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,00,A07-01-00,A12-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,01,A07-01-01,A12-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,03,A07-01-03,A12-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-03,A01-00,A07-01,00,A07-01-00,A12-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-03,A01-00,A07-01,01,A07-01-01,A12-00
...,...,...,...,...,...,...,...,...,...,...,...
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-09
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-11
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-13
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-14


### 10. 직장

#### i) 직장 전처리

In [ ]:
# 전처리 (1)

def t_job(dfcolumn):

  dfcolumn=dfcolumn.astype('str')

  c_student=['대학','대학원']
  seeker=['취업', '채용', '능력', '구직', '일거리', '일자리', '조사단','퇴사']
  employee=['근로','직장', '임금', '휴직']
  changup=['창업']
  s_business=['중소']
  s_s_business=['영세','소상']
  freelancer=['프리랜서','프리렌서','프리 렌서','프리 랜서']

  f_list=[]
  for _ in dfcolumn:
    list=[]
    if any(str in _ for str in c_student):
      list.append('A04-01')
    if any(str in _ for str in seeker):
      list.append('A04-02')
    if any(str in _ for str in employee):
      list.append('A04-03')
    if any(str in _ for str in changup):
      list.append('A04-04')
    if any(str in _ for str in s_s_business):
      list.append('A04-05')
    if any(str in _ for str in s_business):
      list.append('A04-06')
    if any(str in _ for str in freelancer):
      list.append('A04-07')
    if len(list)==0:
      list.append('A04-00')
    f_list.append(','.join(list))
  return f_list

In [ ]:
df['태그-직장']=t_job(df['Full'])
df['태그-직장'].head()

0           A04-00
1           A04-03
2    A04-02,A04-03
3           A04-06
4           A04-00
Name: 태그-직장, dtype: object

In [ ]:
# 포함 항목이 4개 이상이면 '무관'으로 분류

for i in range (len(df)):
  if len(df['태그-직장'][i]) >= 27 :
    df['태그-직장'][i] = 'A04-00'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
df['태그-직장'].value_counts()

A04-00                  66874
A04-04                   4584
A04-02                   3572
A04-02,A04-03            3185
A04-04,A04-05,A04-06     2887
A04-03                   2564
A04-02,A04-04            1116
A04-01                    680
A04-03,A04-04             419
A04-02,A04-03,A04-04      414
A04-06                    328
A04-05,A04-06             297
A04-01,A04-02,A04-03      267
A04-04,A04-05             253
A04-03,A04-06             236
A04-01,A04-04             208
A04-01,A04-02             160
A04-04,A04-06              74
A04-05                     71
A04-02,A04-05,A04-06       51
A04-01,A04-02,A04-04       45
A04-02,A04-03,A04-06       29
A04-03,A04-05,A04-06       28
A04-01,A04-03              23
A04-03,A04-05              18
A04-02,A04-06              15
A04-03,A04-06,A04-07       12
A04-02,A04-03,A04-05        9
A04-01,A04-06               8
A04-02,A04-04,A04-06        7
A04-02,A04-05               7
A04-01,A04-02,A04-06        6
A04-02,A04-04,A04-05        5
A04-03,A04

#### ii) 직장상세 전처리

In [ ]:
keyword1 = ['건설', '채굴','경영', '사무', '금융', '보험','교육', '법률', '사회복지', '경찰', '소방', '군인', '공무원','농업','어업','미용','여행','숙박','음식','경비','돌봄','청소','보건', '의료','설비','정비','생산','연구','공학','영업','판매','운전','운송','예술','디자인','방송','스포츠']
keyword2 = ['농업','축산업','어업','공공','행정','국방','방위','보건','사회복지','광업','임업','교육','서비스','전문','과학','기술','제조','협회','단체','출판','방송','정보서비스','사업시설관리','지역개발','건설','전기','가스', '수도','예술','스포츠','여가','도매', '소매','숙박','음식','음식점','운수','부동산', '임대','금융', '보험','하수', '폐기', '환경']

In [ ]:
# 전처리 (2)

def t_target_work_detail(dfcolumn):

  dfcolumn=dfcolumn.astype('str')
  dfcolumn2=dfcolumn['서비스목적'] + " " +dfcolumn['지원내용'] + " " + dfcolumn['지원대상']

  f_list=[]
  for _ in range(0,dfcolumn.shape[0]):

    list=[]

    if 'A04-00' in dfcolumn['태그-직장'][_]:
      list.append('A04-00')  

    if 'A04-01' in dfcolumn['태그-직장'][_]:
      if any(str in dfcolumn2[_] for str in ['대학원']) == False:
        list.append('A04-01-00')
      if any(str in dfcolumn2[_] for str in ['대학원']):
        list.append('A04-01-01')
    
    if 'A04-02' in dfcolumn['태그-직장'][_]:
      if any(str in dfcolumn2[_] for str in ['건설', '채굴']):
        list.append('A04-02-00')
      if any(str in dfcolumn2[_] for str in ['경영', '사무', '금융', '보험']):
        list.append('A04-02-01')
      if any(str in dfcolumn2[_] for str in ['교육', '법률', '사회복지', '경찰', '소방', '군인', '공무원']):
        list.append('A04-02-02')
      if any(str in dfcolumn2[_] for str in ['농업','어업']):
        list.append('A04-02-03')
      if any(str in dfcolumn2[_] for str in ['미용','여행','숙박','음식','경비','돌봄','청소']):
        list.append('A04-02-04')
      if any(str in dfcolumn2[_] for str in ['보건', '의료']) :
        list.append('A04-02-05')
      if any(str in dfcolumn2[_] for str in ['설비','정비','생산']):
        list.append('A04-02-06')
      if any(str in dfcolumn2[_] for str in ['연구','공학']):
        list.append('A04-02-07')
      if any(str in dfcolumn2[_] for str in ['영업','판매','운전','운송']):
        list.append('A04-02-08')
      if any(str in dfcolumn2[_] for str in ['예술','디자인','방송','스포츠']):
        list.append('A04-02-09')
      if any(str in dfcolumn2[_] for str in keyword1) == False:
        list.append('A04-02')

    if 'A04-03' in dfcolumn['태그-직장'][_]:
      if any(str in dfcolumn2[_] for str in ['건설', '채굴']):
        list.append('A04-03-00')
      if any(str in dfcolumn2[_] for str in ['경영', '사무', '금융', '보험']):
        list.append('A04-03-01')
      if any(str in dfcolumn2[_] for str in ['교육', '법률', '사회복지', '경찰', '소방', '군인', '공무원']):
        list.append('A04-03-02')
      if any(str in dfcolumn2[_] for str in ['농업','어업']):
        list.append('A04-03-03')
      if any(str in dfcolumn2[_] for str in ['미용','여행','숙박','음식','경비','돌봄','청소']):
        list.append('A04-03-04')
      if any(str in dfcolumn2[_] for str in ['보건', '의료']) :
        list.append('A04-03-05')
      if any(str in dfcolumn2[_] for str in ['설비','정비','생산']):
        list.append('A04-03-06')
      if any(str in dfcolumn2[_] for str in ['연구','공학']):
        list.append('A04-03-07')
      if any(str in dfcolumn2[_] for str in ['영업','판매','운전','운송']):
        list.append('A04-03-08')
      if any(str in dfcolumn2[_] for str in ['예술','디자인','방송','스포츠']):
        list.append('A04-03-09')
      if any(str in dfcolumn2[_] for str in keyword1) == False:
        list.append('A04-03')
    
    if 'A04-04' in dfcolumn['태그-직장'][_]:
      if any(str in dfcolumn2[_] for str in ['농업','축산업','어업']):
        list.append('A04-04-00')
      if any(str in dfcolumn2[_] for str in ['공공','행정','국방','방위']):
        list.append('A04-04-01')
      if any(str in dfcolumn2[_] for str in ['보건','사회복지']):
        list.append('A04-04-02')
      if any(str in dfcolumn2[_] for str in ['광업','임업']):
        list.append('A04-04-03')
      if any(str in dfcolumn2[_] for str in ['교육']):
        list.append('A04-04-04')
      if any(str in dfcolumn2[_] for str in ['서비스']) :
        list.append('A04-04-05')
      if any(str in dfcolumn2[_] for str in ['전문','과학','기술']):
        list.append('A04-04-06')
      if any(str in dfcolumn2[_] for str in ['제조']):
        list.append('A04-04-07')
      if any(str in dfcolumn2[_] for str in ['협회','단체']):
        list.append('A04-04-08')
      if any(str in dfcolumn2[_] for str in ['출판','방송','정보서비스']):
        list.append('A04-04-09')
      if any(str in dfcolumn2[_] for str in ['사업시설관리','지역개발']):
        list.append('A04-04-10')
      if any(str in dfcolumn2[_] for str in ['건설']):
        list.append('A04-04-11')
      if any(str in dfcolumn2[_] for str in ['전기','가스', '수도']):
        list.append('A04-04-12')
      if any(str in dfcolumn2[_] for str in ['예술','스포츠','여가']):
        list.append('A04-04-13')
      if any(str in dfcolumn2[_] for str in ['도매', '소매']) :
        list.append('A04-04-14')
      if any(str in dfcolumn2[_] for str in ['숙박','음식','음식점']):
        list.append('A04-04-15')
      if any(str in dfcolumn2[_] for str in ['운수']):
        list.append('A04-04-16')
      if any(str in dfcolumn2[_] for str in ['부동산', '임대']):
        list.append('A04-04-17')
      if any(str in dfcolumn2[_] for str in ['금융', '보험']):
        list.append('A04-04-18')
      if any(str in dfcolumn2[_] for str in ['하수', '폐기', '환경']):
        list.append('A04-04-19')
      if any(str in dfcolumn2[_] for str in keyword2) == False:
        list.append('A04-04')

    if 'A04-05' in dfcolumn['태그-직장'][_]:
      if any(str in dfcolumn2[_] for str in ['농업','축산업','어업']):
        list.append('A04-05-00')
      if any(str in dfcolumn2[_] for str in ['공공','행정','국방','방위']):
        list.append('A04-05-01')
      if any(str in dfcolumn2[_] for str in ['보건','사회복지']):
        list.append('A04-05-02')
      if any(str in dfcolumn2[_] for str in ['광업','임업']):
        list.append('A04-05-03')
      if any(str in dfcolumn2[_] for str in ['교육']):
        list.append('A04-05-04')
      if any(str in dfcolumn2[_] for str in ['서비스']) :
        list.append('A04-05-05')
      if any(str in dfcolumn2[_] for str in ['전문','과학','기술']):
        list.append('A04-05-06')
      if any(str in dfcolumn2[_] for str in ['제조']):
        list.append('A04-05-07')
      if any(str in dfcolumn2[_] for str in ['협회,단체']):
        list.append('A04-05-08')
      if any(str in dfcolumn2[_] for str in ['출판','방송','정보서비스']):
        list.append('A04-05-09')
      if any(str in dfcolumn2[_] for str in ['사업시설관리업','지역개발사업']):
        list.append('A04-05-10')
      if any(str in dfcolumn2[_] for str in ['건설']):
        list.append('A04-05-11')
      if any(str in dfcolumn2[_] for str in ['전기','가스', '수도']):
        list.append('A04-05-12')
      if any(str in dfcolumn2[_] for str in ['예술','스포츠','여가']):
        list.append('A04-05-13')
      if any(str in dfcolumn2[_] for str in ['도매', '소매']) :
        list.append('A04-05-14')
      if any(str in dfcolumn2[_] for str in ['숙박','음식','음식점']):
        list.append('A04-05-15')
      if any(str in dfcolumn2[_] for str in ['운수']):
        list.append('A04-05-16')
      if any(str in dfcolumn2[_] for str in ['부동산', '임대']):
        list.append('A04-05-17')
      if any(str in dfcolumn2[_] for str in ['금융', '보험']):
        list.append('A04-05-18')
      if any(str in dfcolumn2[_] for str in ['하수', '폐기', '환경']):
        list.append('A04-05-19')
      if any(str in dfcolumn2[_] for str in keyword2) == False:
        list.append('A04-05')

    if 'A04-06' in dfcolumn['태그-직장'][_]:
      if any(str in dfcolumn2[_] for str in ['농업','축산업','어업']):
        list.append('A04-06-00')
      if any(str in dfcolumn2[_] for str in ['공공','행정','국방','방위']):
        list.append('A04-06-01')
      if any(str in dfcolumn2[_] for str in ['보건','사회복지']):
        list.append('A04-06-02')
      if any(str in dfcolumn2[_] for str in ['광업','임업']):
        list.append('A04-06-03')
      if any(str in dfcolumn2[_] for str in ['교육']):
        list.append('A04-06-04')
      if any(str in dfcolumn2[_] for str in ['서비스']) :
        list.append('A04-06-05')
      if any(str in dfcolumn2[_] for str in ['전문','과학','기술']):
        list.append('A04-06-06')
      if any(str in dfcolumn2[_] for str in ['제조']):
        list.append('A04-06-07')
      if any(str in dfcolumn2[_] for str in ['협회,단체']):
        list.append('A04-06-08')
      if any(str in dfcolumn2[_] for str in ['출판','방송','정보서비스']):
        list.append('A04-06-09')
      if any(str in dfcolumn2[_] for str in ['사업시설관리업','지역개발사업']):
        list.append('A04-06-10')
      if any(str in dfcolumn2[_] for str in ['건설']):
        list.append('A04-06-11')
      if any(str in dfcolumn2[_] for str in ['전기','가스', '수도']):
        list.append('A04-06-12')
      if any(str in dfcolumn2[_] for str in ['예술','스포츠','여가']):
        list.append('A04-06-13')
      if any(str in dfcolumn2[_] for str in ['도매', '소매']) :
        list.append('A04-06-14')
      if any(str in dfcolumn2[_] for str in ['숙박','음식','음식점']):
        list.append('A04-06-15')
      if any(str in dfcolumn2[_] for str in ['운수']):
        list.append('A04-06-16')
      if any(str in dfcolumn2[_] for str in ['부동산', '임대']):
        list.append('A04-06-17')
      if any(str in dfcolumn2[_] for str in ['금융', '보험']):
        list.append('A04-06-18')
      if any(str in dfcolumn2[_] for str in ['하수', '폐기', '환경']):
        list.append('A04-06-19')
      if any(str in dfcolumn2[_] for str in keyword2) == False:
        list.append('A04-06')

    if 'A04-07' in dfcolumn['태그-직장'][_]:
      list.append('A04-07')

    f_list.append(','.join(list))

  return f_list

In [ ]:
df['태그-직장상세'] = t_target_work_detail(df)

In [ ]:
# 너무 많은 항목이 해당될 경우, '무관'으로 바꾸기

for i in range(len(df)):
  if len(str(df['태그-직장상세'][i])) > 150 :
    df['태그-직장상세'][i] = 'A04-00'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
df['태그-직장상세'].value_counts()

A04-00                                                                                                 66912
A04-04-00                                                                                               1644
A04-02-02,A04-02-05                                                                                      744
A04-02-02                                                                                                731
A04-02-02,A04-03-02                                                                                      697
                                                                                                       ...  
A04-02-01,A04-02-02,A04-02-04,A04-02-05,A04-02-09,A04-03-01,A04-03-02,A04-03-04,A04-03-05,A04-03-09        1
A04-02-01,A04-02-02,A04-02-06,A04-04-18                                                                    1
A04-04-00,A04-04-03,A04-04-05,A04-04-17,A04-05-00,A04-05-03,A04-05-05,A04-05-17                            1
A04-05-13,A04-05-19

In [ ]:
worked = df['태그-직장상세'].str.replace(',','/')
worked_list = worked.str.split('/')
worked_data = worked_list.apply(lambda x : pd.Series(x))
workedframe = worked_data.stack().reset_index(level=1, drop=True).to_frame('태그-직장상세목록')

In [ ]:
df_1 = df_1.merge(workedframe, left_index=True, right_index=True, how='left')

In [ ]:
df_1

,태그-신청절차,태그-가구원목록,태그-성별,태그-학력목록,태그-결혼,태그-관심상황목록,지역_number,태그-자녀,태그-자녀상세목록,자녀_number,태그-대상특성상세목록,태그-직장상세목록
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,00,A07-01-00,A12-00,A04-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,01,A07-01-01,A12-00,A04-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,03,A07-01-03,A12-00,A04-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-03,A01-00,A07-01,00,A07-01-00,A12-00,A04-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-03,A01-00,A07-01,01,A07-01-01,A12-00,A04-00
...,...,...,...,...,...,...,...,...,...,...,...,...
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-09,A04-00
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-11,A04-00
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-13,A04-00
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-14,A04-00


### 11. 지원유형

#### i) 지원유형 전처리

In [ ]:
# 전처리 (1)

def t_apply_type(dfcolumn):
  dfcolumn=dfcolumn.astype('str')

  f_list=[]

  for _ in dfcolumn:
    list=[]
    if '현금' in _:
      list.append('A09-00')
    if '현물' in _:
      list.append('A09-01')
    if '이용권' in _:
      list.append('A09-02')
    if '서비스' in _:
      list.append('A09-03')
    if '의료지원' in _:
      list.append('A09-04')
    if '상담/법률지원' in _:
      list.append('A09-05')
    if '문화/여가지원' in _:
      list.append('A09-06')
    if '민원' in _:
      list.append('A09-07')
    if '봉사/기부' in _:
      list.append('A09-08')

    if len(list)==0:
      list.append('A09-09')
    f_list.append(','.join(list))
  return f_list

In [ ]:
df['태그-지원유형']=t_apply_type(df['지원유형'])
df['태그-지원유형'].head()

0           A09-00
1           A09-00
2    A09-00,A09-05
3           A09-00
4           A09-00
Name: 태그-지원유형, dtype: object

In [ ]:
df['태그-지원유형'].value_counts()

A09-00,A09-01                                29113
A09-09                                       22307
A09-00,A09-01,A09-04                          9195
A09-06                                        4957
A09-04                                        4522
                                             ...  
A09-02,A09-03,A09-04,A09-05,A09-06,A09-07        1
A09-01,A09-03,A09-04                             1
A09-02,A09-03,A09-06,A09-07                      1
A09-00,A09-01,A09-03                             1
A09-02,A09-03,A09-05,A09-06,A09-08               1
Name: 태그-지원유형, Length: 134, dtype: int64

#### ii) 지원유형상세 전처리

In [ ]:
# 전처리 (2)

def t_target_type_detail(dfcolumn):

  dfcolumn=dfcolumn.astype('str')
  dfcolumn2 = dfcolumn['서비스목적'] + " " +dfcolumn['지원내용'] + " " + dfcolumn['지원대상']

  f_list=[]
  for _ in range(0,dfcolumn.shape[0]):

    list=[]

    if 'A09-00' in dfcolumn['태그-지원유형'][_]:
      if any(str in dfcolumn2[_] for str in ['장학금']):
        list.append('A09-00-01')
      if any(str in dfcolumn2[_] for str in ['융자']):
        list.append('A09-00-02')
      if any(str in dfcolumn2[_] for str in ['감면']):
        list.append('A09-00-03')
      if any(str in dfcolumn2[_] for str in ['보험']):
        list.append('A09-00-04')
      if any(str in dfcolumn2[_] for str in ['장학금', '융자', '감면', '보험']) == False:
        list.append('A09-00-00')
    
    if 'A09-01' in dfcolumn['태그-지원유형'][_]:
      list.append('A09-01')

    if 'A09-02' in dfcolumn['태그-지원유형'][_]:
      list.append('A09-02')

    if 'A09-03' in dfcolumn['태그-지원유형'][_]:
      if any(str in dfcolumn2[_] for str in ['임대']):
        list.append('A09-03-01')
      if any(str in dfcolumn2[_] for str in ['일자리', '일']):
        list.append('A09-03-02')
      if any(str in dfcolumn2[_] for str in ['의료', '치료']):
        list.append('A09-03-03')
      if any(str in dfcolumn2[_] for str in ['돌봄', '보육']):
        list.append('A09-03-04')
      if any(str in dfcolumn2[_] for str in ['임대','일자리','일','의료','치료','돌봄', '보육']) == False:
        list.append('A09-03-00')

    if 'A09-04' in dfcolumn['태그-지원유형'][_]:
      list.append('A09-04')

    if 'A09-05' in dfcolumn['태그-지원유형'][_]:
      list.append('A09-05')

    if 'A09-06' in dfcolumn['태그-지원유형'][_]:
      list.append('A09-06')

    if 'A09-07' in dfcolumn['태그-지원유형'][_]:
      list.append('A09-07')

    if 'A09-08' in dfcolumn['태그-지원유형'][_]:
      list.append('A09-08')

    if 'A09-09' in dfcolumn['태그-지원유형'][_]:
      if any(str in dfcolumn2[_] for str in ['교육']):
        list.append('A09-09-01')
      if any(str in dfcolumn2[_] for str in ['상담']):
        list.append('A09-09-02')
      if any(str in dfcolumn2[_] for str in ['교육', '상담'])==False:
        list.append('A09-09-00')

    f_list.append(','.join(list))

  return f_list

In [ ]:
df['태그-지원유형상세'] = t_target_type_detail(df)

In [ ]:
apply = df['태그-지원유형상세'].str.replace(',','/')
apply_list = apply.str.split('/')
apply_data = apply_list.apply(lambda x : pd.Series(x))
applyframe = apply_data.stack().reset_index(level=1, drop=True).to_frame('태그-지원유형상세목록')

In [ ]:
df_1 = df_1.merge(applyframe, left_index=True, right_index=True, how='left')

In [ ]:
df_1

,태그-신청절차,태그-가구원목록,태그-성별,태그-학력목록,태그-결혼,태그-관심상황목록,지역_number,태그-자녀,태그-자녀상세목록,자녀_number,태그-대상특성상세목록,태그-직장상세목록,태그-지원유형상세목록
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,00,A07-01-00,A12-00,A04-00,A09-00-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,01,A07-01-01,A12-00,A04-00,A09-00-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,03,A07-01-03,A12-00,A04-00,A09-00-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-03,A01-00,A07-01,00,A07-01-00,A12-00,A04-00,A09-00-00
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-03,A01-00,A07-01,01,A07-01-01,A12-00,A04-00,A09-00-00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-09,A04-00,A09-06
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-11,A04-00,A09-06
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-13,A04-00,A09-06
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-14,A04-00,A09-06


In [ ]:
df['태그-대상연령시작'].value_counts()

0     70943
6      7891
65     5976
13     3248
19      346
50       67
30       15
Name: 태그-대상연령시작, dtype: int64

In [ ]:
df['태그-대상연령끝'].value_counts()

99    74127
12     8244
18     3743
5      1920
29      355
64       78
49       19
Name: 태그-대상연령끝, dtype: int64

In [ ]:
df_1['age_start'] = df['태그-대상연령시작']

In [ ]:
df_1['age_end'] = df['태그-대상연령끝']

In [ ]:
df['태그-중위소득시작'].value_counts()

0    88486
Name: 태그-중위소득시작, dtype: int64

In [ ]:
df

,정책ID,정책서비스ID,서비스명,소관기관,소관기관유형,소관기관.1,문의처전화번호,소관기관유형.1,온라인신청사이트URL,생애주기,신청절차,선정기준,지원유형,서비스목적,지원내용,지원대상,신청기한_시작,신청기한_끝,상세조회URL,태그-신청절차,태그-자녀,태그-수집유형,태그-가구원,태그-성별,태그-학력,태그-직장,태그-결혼,태그-지원유형,태그-지역(시도),태그-관심상황특성,태그-자녀상세,태그-직장상세,태그-지원유형상세,태그-지역(시군구),태그-대상특성상세,태그-대상특성,태그-대상연령시작,태그-대상연령끝,태그-중위소득시작,태그-중위소득끝,Full
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),교육부,중앙행정기관,교육부,129,중앙정부,http://online.bokjiro.go.kr/,영·유아||아동·청소년||청년||중장년||노년기,신청ㆍ접수(읍면동사무소) - 보장 결정(시군구 사업팀),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,현금,누리과정이란 만 3~5세를 대상으로 하는 어린이집 표준보육과정과 유치원 교육과정을 ...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/00000046...,오프라인,A07-01,NaN,무관,무관,무관,A04-00,A06-02,A09-00,A01-00,"A13-01,A13-03","00,01,03",A04-00,A09-00-00,,A12-00,A12-00,6,12,0,0,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영) 1) 지원대상 : 어린...
1,2,105100000001,근로·자녀장려금,국세청,중앙행정기관,국세청,NaN,중앙정부,https://www.hometax.go.kr,NaN,"○ 개벌 신청 안내받은 경우에는 ARS, 모바일 국세청 홈택스 앱, 인터넷 홈택스 ...","○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",현금,○ 근로자녀장려세제\n - 저소득계층의 빈곤 탈출을 지원하고 사회안전망을 확충하기 ...,○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/10510000...,온라인,A07-01,NaN,주택 세대원,무관,무관,A04-03,A06-02,A09-00,A01-00,A13-03,,A04-03,A09-00-00,,A12-00,A12-00,0,99,0,0,"근로·자녀장려금 ○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 ..."
2,3,116010000001,월세 자금보증,금융위원회,중앙행정기관,금융위원회,-,중앙정부,-,NaN,"신청 방식 : 은행 방문\n신청 절차\n1. 은행 방문 및 상담 신청\n2. 보증,...",○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,상담/법률지원||현금(융자),저소득층 등에게 월세자금보증을 하여 저소득층의 주거 안정성 제고,○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원\...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11601000...,오프라인,A07-00,NaN,"주택 세대주,무주택 세대주",무관,무관,"A04-02,A04-03",A06-02,"A09-00,A09-05",A01-00,A13-04,,"A04-02,A04-03","A09-00-00,A09-05",,A12-00,A12-00,0,99,0,0,월세 자금보증 ○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대...
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),금융위원회,중앙행정기관,금융위원회,1588-1500,중앙정부,http://www.kdb.co.kr,NaN,○ 3.26일부터 기업의 신청접수ㆍ지원 중\n\n ➡ 산업은행 전국 영업점을 방문하...,○ 산업은행 심사기준 충족 업체,현금(융자),코로나19로 피해를 입은 중소ㆍ중견기업 지원 프로그램,○ 자금용도 : 운영자금\n\n○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부...,"○ 지원대상 : 코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 ...",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11601000...,오프라인,A07-00,NaN,무관,무관,무관,A04-06,A06-00,A09-00,A01-00,A13-06,,A04-06,A09-00-00,,"A12-03-00,A12-19","A12-03,A12-19",0,99,0,0,‘힘내라 대한민국’특별운영자금 (산업은행) ○ 산업은행 심사기준 충족 업체 코로나1...
4,5,119200000001,친환경 에너지 절감 장비 보급,해양수산부,중앙행정기관,해양수산부,044-200-5526,중앙정부,-,NaN,해당 지자체(시군구청)에 방문하여 신청,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우...",현금,연근해 어업인을 대상으로 노후기관ㆍ장비ㆍ설비 대체 보급으로 안정적 조업기반 조성,○ 유류비 절감을 위한 유류절감장치 설치\n○ 저효율(육상기관 포함)기관 및 사...,"○ 어선 노후기관, 장비를 교체하고자 하는 연근해 어업인",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11920000...,오프라인,A07-00,NaN,무관,무관,무관,A04-00,A06-00,A09-00,A01-00,A13-03,,A04-00,A09-00-00,,"A12-02-06,A12-02-08,A12-19","A12-02,A12-19",0,99,0,0,친환경 에너지 절감 장비 보급 ○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88481,88482,645000000579,FTA 피해 보전 직불금 지원,전라북도,광역시도,전라북도,주민센터,광역시도,NaN,청년||중장년||노년기,신청 방식-신청 서류를 작성하여 해당 시군구(읍면동)에 방문 또는 우편접수\n신청 ...,○ 다음 각 요건을 모두 충족한 자\n -농업인 등에 해당하는 자로 ｢농어업경영체 ...,현금/현물,FTA 이행으로 수입이 급증하여 가격 하락의 피해를 입은 품목의 생산자에게 가격 하...,"FTA 협정 이행으로 인한 수입 증가로 가격 하락의 피해가 발생할 경우, 발생된 피...",○ FTA 피해 보전직불금 지원 대상 품목을 해당 협정의 발효일 이전부터 생산한 농...,NaN,NaN,NaN,복합,A07-00,NaN,무관,무관,무관,A04-04,A06-00,"A09-00,A09-01",A01-13,A13-05,,A04-04-00,"A09-00-00,A09-01",00,A12-02-10,A12-02,0,99,0,0,FTA 피해 보전 직불금 지원 ○ 다음 각 요건을 모두 충족한 자\n -농업인 등에...
88482,88483,650000000822,이혼위기가족지원,제주특별자치도,광역시도,제주특별자치도,064-725-8030,광역시도,http://www.familynet.or.kr/fn_areacenter/progr...,청년||중장년||노년기,"건강가정 지원센터 중 사업 수행기관 방문, 전화",협의 및 재판 이혼 신청자 등 이혼을 준비 중인 부부와 아동,상담/법률지원||문화/여가지원\

In [ ]:
df['income_target_text']=df['선정기준']+' '+df['지원대상']

In [ ]:
df['태그-중위소득시작']=0
df['태그-중위소득끝']=999

In [ ]:
import re

In [ ]:
for a in range(0,df['income_target_text'].shape[0]):
  
  g=df['income_target_text'][a]

  start_num=0
  end_num=999

  target = '소득'
  index = -1
  t_list=[]

  while True:
    index = str(g).find(target, index + 1)
    if index == -1:
      break
    t_list.append(index)
  
  low_income='차상위','기초생활','저소득층'
  if any(n in str(g) for n in low_income):
    end_num=50
  if (0< end_num) and (end_num < 999):
    df['태그-중위소득끝'][a]=end_num
      
  else:
    for i in t_list:
      if '~' in g[i:i+15]:
        g1=list(g[i:i+15])
        for i in range(len(g1)):
          g1[i] = str(g1[i])
        g1="".join(g1)
        if "".join(re.findall("\d",  g1[g1.find('~')-5:g1.find('~')])) != '':
          start_num=int("".join(re.findall("\d",  g1[g1.find('~')-5:g1.find('~')])))
        if ("".join(re.findall("\d",g1[g1.find('~'):g1.find('~')+5])) != '') and (end_num < 999):
          end_num=int("".join(re.findall("\d",g1[g1.find('~'):g1.find('~')+5])))

      if '이상' in g[i:i+15]:
        g1=list(g[i:i+15])
        for i in range(len(g1)):
          g1[i] = str(g1[i])
        g1="".join(g1)
        if "".join(re.findall("\d", g1[g1.find('이상')-5:g1.find('이상')])) != '':
          start_num=int("".join(re.findall("\d", g1[g1.find('이상')-5:g1.find('이상')])))
      
      if '초과' in g[i:i+15]:
        g1=list(g[i:i+15])
        for i in range(len(g1)):
          g1[i] = str(g1[i])
        g1="".join(g1)
        if "".join(re.findall("\d", g1[g1.find('초과')-5:g1.find('초과')])) != '':
          start_num=int("".join(re.findall("\d", g1[g1.find('초과')-5:g1.find('초과')])))
      
      if '이하' in g[i:i+15]:
        g1=list(g[i:i+15])
        for i in range(len(g1)):
          g1[i] = str(g1[i])
        g1="".join(g1)
        if ("".join(re.findall("\d", g1[g1.find('이하')-5:g1.find('이하')])) != '') and (end_num < 999):
          end_num=int("".join(re.findall("\d", g1[g1.find('이하')-5:g1.find('이하')])))

      if '미만' in g[i:i+15]:
        g1=list(g[i:i+15])
        for i in range(len(g1)):
          g1[i] = str(g1[i])
        g1="".join(g1)
        if ("".join(re.findall("\d", g1[g1.find('미만')-5:g1.find('미만')])) != '') and (end_num < 999):
          end_num=int("".join(re.findall("\d", g1[g1.find('미만')-5:g1.find('미만')])))

      if start_num > 0:
        df['태그-중위소득시작'][a]=start_num
      
      if (0< end_num) and (end_num < 999):
        df['태그-중위소득끝'][a]=end_num
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
print(df['태그-중위소득시작'].value_counts())
print('--------------------------------------------------')
print(df['태그-중위소득끝'].value_counts())

0      88359
50        32
7         28
100       15
80         8
150        7
1          6
65         6
120        4
60         3
18         3
12         2
6          2
3          2
20         1
30         1
35         1
16         1
40         1
70         1
4          1
180        1
53         1
Name: 태그-중위소득시작, dtype: int64
--------------------------------------------------
999    72250
50     16236
Name: 태그-중위소득끝, dtype: int64


In [ ]:
df

,정책ID,정책서비스ID,서비스명,소관기관,소관기관유형,소관기관.1,문의처전화번호,소관기관유형.1,온라인신청사이트URL,생애주기,신청절차,선정기준,지원유형,서비스목적,지원내용,지원대상,신청기한_시작,신청기한_끝,상세조회URL,태그-신청절차,태그-자녀,태그-수집유형,태그-가구원,태그-성별,태그-학력,태그-직장,태그-결혼,태그-지원유형,태그-지역(시도),태그-관심상황특성,태그-자녀상세,태그-직장상세,태그-지원유형상세,태그-지역(시군구),태그-대상특성상세,태그-대상특성,태그-대상연령시작,태그-대상연령끝,태그-중위소득시작,태그-중위소득끝,Full,income_target_text
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),교육부,중앙행정기관,교육부,129,중앙정부,http://online.bokjiro.go.kr/,영·유아||아동·청소년||청년||중장년||노년기,신청ㆍ접수(읍면동사무소) - 보장 결정(시군구 사업팀),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,현금,누리과정이란 만 3~5세를 대상으로 하는 어린이집 표준보육과정과 유치원 교육과정을 ...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/00000046...,오프라인,A07-01,NaN,무관,무관,무관,A04-00,A06-02,A09-00,A01-00,"A13-01,A13-03","00,01,03",A04-00,A09-00-00,,A12-00,A12-00,6,12,0,999,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영) 1) 지원대상 : 어린...,1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...
1,2,105100000001,근로·자녀장려금,국세청,중앙행정기관,국세청,NaN,중앙정부,https://www.hometax.go.kr,NaN,"○ 개벌 신청 안내받은 경우에는 ARS, 모바일 국세청 홈택스 앱, 인터넷 홈택스 ...","○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",현금,○ 근로자녀장려세제\n - 저소득계층의 빈곤 탈출을 지원하고 사회안전망을 확충하기 ...,○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/10510000...,온라인,A07-01,NaN,주택 세대원,무관,무관,A04-03,A06-02,A09-00,A01-00,A13-03,,A04-03,A09-00-00,,A12-00,A12-00,0,99,0,999,"근로·자녀장려금 ○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 ...","○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\..."
2,3,116010000001,월세 자금보증,금융위원회,중앙행정기관,금융위원회,-,중앙정부,-,NaN,"신청 방식 : 은행 방문\n신청 절차\n1. 은행 방문 및 상담 신청\n2. 보증,...",○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,상담/법률지원||현금(융자),저소득층 등에게 월세자금보증을 하여 저소득층의 주거 안정성 제고,○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원\...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11601000...,오프라인,A07-00,NaN,"주택 세대주,무주택 세대주",무관,무관,"A04-02,A04-03",A06-02,"A09-00,A09-05",A01-00,A13-04,,"A04-02,A04-03","A09-00-00,A09-05",,A12-00,A12-00,0,99,0,999,월세 자금보증 ○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대...,○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),금융위원회,중앙행정기관,금융위원회,1588-1500,중앙정부,http://www.kdb.co.kr,NaN,○ 3.26일부터 기업의 신청접수ㆍ지원 중\n\n ➡ 산업은행 전국 영업점을 방문하...,○ 산업은행 심사기준 충족 업체,현금(융자),코로나19로 피해를 입은 중소ㆍ중견기업 지원 프로그램,○ 자금용도 : 운영자금\n\n○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부...,"○ 지원대상 : 코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 ...",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11601000...,오프라인,A07-00,NaN,무관,무관,무관,A04-06,A06-00,A09-00,A01-00,A13-06,,A04-06,A09-00-00,,"A12-03-00,A12-19","A12-03,A12-19",0,99,0,999,‘힘내라 대한민국’특별운영자금 (산업은행) ○ 산업은행 심사기준 충족 업체 코로나1...,"○ 산업은행 심사기준 충족 업체 ○ 지원대상 : 코로나19 등 질병, 자연재해 등 ..."
4,5,119200000001,친환경 에너지 절감 장비 보급,해양수산부,중앙행정기관,해양수산부,044-200-5526,중앙정부,-,NaN,해당 지자체(시군구청)에 방문하여 신청,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우...",현금,연근해 어업인을 대상으로 노후기관ㆍ장비ㆍ설비 대체 보급으로 안정적 조업기반 조성,○ 유류비 절감을 위한 유류절감장치 설치\n○ 저효율(육상기관 포함)기관 및 사...,"○ 어선 노후기관, 장비를 교체하고자 하는 연근해 어업인",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11920000...,오프라인,A07-00,NaN,무관,무관,무관,A04-00,A06-00,A09-00,A01-00,A13-03,,A04-00,A09-00-00,,"A12-02-06,A12-02-08,A12-19","A12-02,A12-19",0,99,0,999,친환경 에너지 절감 장비 보급 ○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로...,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88481,88482,645000000579,FTA 피해 보전 직불금 지원,전라북도,광역시도,전라북도,주민센터,광역시도,NaN,청년||중장년||노년기,신청 방식-신청 서류를 작성하여 해당 시군구(읍면동)에 방문 또는 우편접수\n신청 ...,○ 다음 각 요건을 모두 충족한 자\n -농업인 등에 해당하는 자로 ｢농어업경영체 ...,현금/현물,FTA 이행으로 수입이 급증하여 가격 하락의 피해를 입은 품목의 생산자에게 가격 하...,"FTA 협정 이행으로 인한 수입 증가로 가격 하락의 피해가 발생할 경우, 발생된 피...",○ FTA 피해 보전직불금 지원 대상 품목을 해당 협정의 발효일 이전부터 생산한 농...,NaN,NaN,NaN,복합,A07-00,NaN,무관,무관,무관,A04-04,A06-00,"A09-00,A09-01",A01-13,A13-05,,A04-04-00,"A09-00-00,A09-01",00,A12

In [ ]:
df_1['income_first'] = df['태그-중위소득시작']

In [ ]:
df_1['income_last'] = df['태그-중위소득끝']

In [ ]:
df_1

,태그-신청절차,태그-가구원목록,태그-성별,태그-학력목록,태그-결혼,태그-관심상황목록,지역_number,태그-자녀,태그-자녀상세목록,자녀_number,태그-대상특성상세목록,태그-직장상세목록,태그-지원유형상세목록,age_start,age_end,income_first,income_last
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,00,A07-01-00,A12-00,A04-00,A09-00-00,6,12,0,999
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,01,A07-01-01,A12-00,A04-00,A09-00-00,6,12,0,999
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-01,A01-00,A07-01,03,A07-01-03,A12-00,A04-00,A09-00-00,6,12,0,999
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-03,A01-00,A07-01,00,A07-01-00,A12-00,A04-00,A09-00-00,6,12,0,999
0,A10-02,A05-00,A02-00,A03-00,A06-02,A13-03,A01-00,A07-01,01,A07-01-01,A12-00,A04-00,A09-00-00,6,12,0,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-09,A04-00,A09-06,65,99,0,999
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-11,A04-00,A09-06,65,99,0,999
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-13,A04-00,A09-06,65,99,0,999
88485,A10-02,A05-00,A02-00,A03-00,A06-00,A13-05,A01-09-12,A07-01,03,A07-01-03,A12-14,A04-00,A09-06,65,99,0,999


### **최종 마무리**

In [ ]:
# 1. 순서 재조정

df_2 = df_1[['지역_number', '태그-성별', '태그-학력목록', '태그-직장상세목록', '태그-가구원목록', '태그-결혼', '자녀_number', '태그-지원유형상세목록', '태그-신청절차', '태그-대상특성상세목록', '태그-관심상황목록', 'age_start', 'age_end', 'income_first', 'income_last']]

In [ ]:
# 2. 컬럼명 변경

df_2.columns = ['code_region', 'code_gender', 'code_graduate', 'code_job', 'code_family', 'code_marriage', 'code_children', 'code_provision_type', 'code_apply_type', 'code_target', 'code_status', 'age_start', 'age_end', 'income_first', 'income_last']

In [ ]:
# 3. 중복항 제거

df_3 = df_2.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)

In [ ]:
df_3['Policy_ID'] = df['정책ID']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# 최종 데이터 확인

df_3

,code_region,code_gender,code_graduate,code_job,code_family,code_marriage,code_children,code_provision_type,code_apply_type,code_target,code_status,age_start,age_end,income_first,income_last,Policy_ID
0,A01-00,A02-00,A03-00,A04-00,A05-00,A06-02,A07-01-00,A09-00-00,A10-02,A12-00,A13-01,6,12,0,999,1
0,A01-00,A02-00,A03-00,A04-00,A05-00,A06-02,A07-01-01,A09-00-00,A10-02,A12-00,A13-01,6,12,0,999,1
0,A01-00,A02-00,A03-00,A04-00,A05-00,A06-02,A07-01-03,A09-00-00,A10-02,A12-00,A13-01,6,12,0,999,1
0,A01-00,A02-00,A03-00,A04-00,A05-00,A06-02,A07-01-00,A09-00-00,A10-02,A12-00,A13-03,6,12,0,999,1
0,A01-00,A02-00,A03-00,A04-00,A05-00,A06-02,A07-01-01,A09-00-00,A10-02,A12-00,A13-03,6,12,0,999,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88485,A01-09-12,A02-00,A03-00,A04-00,A05-00,A06-00,A07-01-03,A09-06,A10-02,A12-09,A13-05,65,99,0,999,88486
88485,A01-09-12,A02-00,A03-00,A04-00,A05-00,A06-00,A07-01-03,A09-06,A10-02,A12-11,A13-05,65,99,0,999,88486
88485,A01-09-12,A02-00,A03-00,A04-00,A05-00,A06-00,A07-01-03,A09-06,A10-02,A12-13,A13-05,65,99,0,999,88486
88485,A01-09-12,A02-00,A03-00,A04-00,A05-00,A06-00,A07-01-03,A09-06,A10-02,A12-14,A13-05,65,99,0,999,88486


In [ ]:
# 엑셀 변환 (주의 - 오래걸림)

df_3.to_excel('policy_final.xlsx')